In [1]:
import pymongo
import pandas as pd
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
import re
companies = db['companies']
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


In [2]:
def convert_companies(company):
    return {
        'name': company['name'],
        'category_code': company['category_code'],
        'founded_year': company['founded_year'],
        'number_of_employees': company['number_of_employees'],
        'total_money_raised': company['total_money_raised'],
        'offices': company['offices']
               }

cursor = companies.find({"$and": [ { "offices.latitude": {"$exists": True} },{ "offices.longitude": {"$exists": True} }, { "founded_year": {"$gt": 2002} }, {"deadpooled_year": None}  ] },{"_id":0})
geoCompanies = list(map(lambda r: convert_companies(r), cursor))

In [3]:
def convert_offices(company):
    companies = []
    for x in geoCompanies:
        for y in x['offices']:
            z = {
            'name': x['name'],
            'category_code': x['category_code'],
            'founded_year': x['founded_year'],
            'number_of_employees': x['number_of_employees'],
            'total_money_raised': x['total_money_raised'],
            'city': y['city'],
            'description': y['description'],
            'country_code': y['country_code'],
            'latitude': y['latitude'],
            'longitude': y['longitude']}
            companies.append(z)
    return companies
offices = convert_offices(geoCompanies)

In [4]:
df = pd.DataFrame(offices)
df.head()


,category_code,city,country_code,description,founded_year,latitude,longitude,name,number_of_employees,total_money_raised
0,news,San Francisco,USA,None,2004,37.764726,-122.394523,Digg,60.0,$45M
1,web,West Hollywood,USA,Headquarters,2006,34.090368,-118.393064,Geni,18.0,$16.5M
2,web,Menlo Park,USA,None,2003,37.484130,-122.169472,Gizmoz,NaN,$18.1M
3,mobile,Los Angeles,USA,None,2005,34.057498,-118.446596,Helio,NaN,$0
4,music,San Diego,USA,,2006,33.022176,-117.081406,Slacker,NaN,$73.1M


In [5]:
df["Local"] = df["name"] + df["description"]
df['Local'].head(109000)

0                                           NaN
1                              GeniHeadquarters
2                                           NaN
3                                           NaN
4                                       Slacker
5                                      ScribdHQ
6                                           NaN
7                                         Joost
8                                       Twitter
9                          FacebookHeadquarters
10                            FacebookEurope HQ
11                             FacebookNew York
12                                          NaN
13                                       Mahalo
14                                         Veoh
15                                          NaN
16                         AddThisHQ - Virginia
17                       AddThisNew York Office
18                    AddThisLos Angeles Office
19                        AddThisChicago Office
20                  AddThisSan Francisco

In [6]:
def posicion(x):
    points = []
    for i in range(len(x)):
        lat = x.iloc[i]['latitude']
        lon = x.iloc[i]['longitude']
        point = {'type':'Point', 'coordinates': [lon,lat]}
        points.append(point)
    df['points'] = points
    return df['points']
posicion(df)

0       {'type': 'Point', 'coordinates': [-122.394523,...
1       {'type': 'Point', 'coordinates': [-118.393064,...
2       {'type': 'Point', 'coordinates': [-122.169472,...
3       {'type': 'Point', 'coordinates': [-118.446596,...
4       {'type': 'Point', 'coordinates': [-117.081406,...
5       {'type': 'Point', 'coordinates': [-122.404052,...
6       {'type': 'Point', 'coordinates': [-73.985506, ...
7       {'type': 'Point', 'coordinates': [-74.0094471,...
8       {'type': 'Point', 'coordinates': [-122.4169244...
9       {'type': 'Point', 'coordinates': [-122.151801,...
10      {'type': 'Point', 'coordinates': [-6.267494, 5...
11      {'type': 'Point', 'coordinates': [-73.9792469,...
12      {'type': 'Point', 'coordinates': [-122.395289,...
13      {'type': 'Point', 'coordinates': [-118.487267,...
14      {'type': 'Point', 'coordinates': [-117.20834, ...
15      {'type': 'Point', 'coordinates': [-122.409173,...
16      {'type': 'Point', 'coordinates': [-77.245195, ...
17      {'type

In [7]:
def moneylaundry(money):
    res = []
    for x in money:
        x = x.replace('$', '').replace('€', '').replace('£', '').replace('¥', '').replace('C', '').replace('kr', '')
        if 'M' in x:
            x = x.replace('M', '')
            res.append(float(x)*1000000)
        elif 'k' in x:
            x = x.replace('k', '')
            res.append(float(x)*1000)

        elif 'B' in x:
            x = x.replace('B', '')
            res.append(float(x)*1000000000)

        else:
            res.append(float(x))
    return res
df['total_money_raised'] = moneylaundry(df['total_money_raised'])
df.head()

,category_code,city,country_code,description,founded_year,latitude,longitude,name,number_of_employees,total_money_raised,Local,points
0,news,San Francisco,USA,None,2004,37.764726,-122.394523,Digg,60.0,45000000.0,NaN,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,web,West Hollywood,USA,Headquarters,2006,34.090368,-118.393064,Geni,18.0,16500000.0,GeniHeadquarters,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,web,Menlo Park,USA,None,2003,37.484130,-122.169472,Gizmoz,NaN,18100000.0,NaN,"{'type': 'Point', 'coordinates': [-122.169472,..."
3,mobile,Los Angeles,USA,None,2005,34.057498,-118.446596,Helio,NaN,0.0,NaN,"{'type': 'Point', 'coordinates': [-118.446596,..."
4,music,San Diego,USA,,2006,33.022176,-117.081406,Slacker,NaN,73100000.0,Slacker,"{'type': 'Point', 'coordinates': [-117.081406,..."


In [8]:
df.dtypes

category_code           object
city                    object
country_code            object
description             object
founded_year             int64
latitude               float64
longitude              float64
name                    object
number_of_employees    float64
total_money_raised     float64
Local                   object
points                  object
dtype: object

In [10]:
df['category_code'].value_counts()

web                 1842
software            1334
advertising          672
games_video          585
mobile               546
enterprise           493
ecommerce            374
other                359
consulting           335
network_hosting      299
public_relations     244
search               212
cleantech            139
hardware             133
biotech              119
security              76
analytics             74
semiconductor         64
social                56
finance               46
news                  46
music                 43
messaging             36
education             29
photo_video           27
travel                25
health                16
medical               14
transportation        14
sports                10
manufacturing         10
real_estate            9
fashion                8
automotive             6
legal                  5
hospitality            5
design                 5
nonprofit              2
nanotech               2
Name: category_code, dtyp

In [11]:
def success(x):
    counter = 0
    if x == ('education' or 'public_relations'):
        counter += (-5)
    elif x == ('semiconductor' or 'music' or 'automotive'):
        counter += (-4)
    elif x == ('messaging' or 'fashion'):
        counter += (-3)
    elif x == ('nonprofit' or  'finance'):   
        counter += (-2)
    elif x == ('other'):
        counter += (-1)
    elif x == ('consulting' or 'search' or 'travel'):
        counter += (1)
    elif x == ('enterprise' or 'analytics' or 'nanotech' or 'design' or 'transportation'):
        counter += (2)
    elif x == ('social' or 'photo_video'):
        counter += (3)
    elif x == ('ecommerce' or 'biotech' or 'security'):   
        counter += (4)
    elif x == ('legal' or 'real_state'):
        counter += (5)
    elif x == ('web' or 'software' or 'network_hosting' or 'cleantech' or 'medical' or 'health' or 'manufacturing' or 'sports' or 'software'):
        counter += (6)
    elif x == ('advertising' or 'hardware'):
        counter += (7)
    elif x == ('hospitality' or 'news'):   
        counter += (8)
    elif x == ('mobile'):
        counter += (9)
    else:
        counter += (10)
    return counter
successpercentage = df['category_code'].map(success)

In [12]:
def power(x):
    budget = 0
    if x >= 800:
        budget += 6
    elif x >= 500 and x < 800:    
        budget += 4
    elif x >= 100 and x < 500:    
        budget += 2
    elif x >= 50 and x < 100:    
        budget -= 1
    else:    
        budget -= 3
    return budget

powermoves = df['number_of_employees'].map(power)

In [13]:
def statevalue(x):
    return x['category_code'].map(success) + x['number_of_employees'].map(power)

df['Value'] = statevalue(df)

In [14]:
df.head()

,category_code,city,country_code,description,founded_year,latitude,longitude,name,number_of_employees,total_money_raised,Local,points,Value
0,news,San Francisco,USA,None,2004,37.764726,-122.394523,Digg,60.0,45000000.0,NaN,"{'type': 'Point', 'coordinates': [-122.394523,...",9
1,web,West Hollywood,USA,Headquarters,2006,34.090368,-118.393064,Geni,18.0,16500000.0,GeniHeadquarters,"{'type': 'Point', 'coordinates': [-118.393064,...",3
2,web,Menlo Park,USA,None,2003,37.484130,-122.169472,Gizmoz,NaN,18100000.0,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",3
3,mobile,Los Angeles,USA,None,2005,34.057498,-118.446596,Helio,NaN,0.0,NaN,"{'type': 'Point', 'coordinates': [-118.446596,...",6
4,music,San Diego,USA,,2006,33.022176,-117.081406,Slacker,NaN,73100000.0,Slacker,"{'type': 'Point', 'coordinates': [-117.081406,...",7


In [15]:
df.to_json('comp_with_coords.json', orient='records', lines= True)